<a href="https://colab.research.google.com/github/soupsoupr/StrokeSegmentation/blob/main/testOnCNNsConstruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nibabel as nib
import os
# Set the file path to the directory containing the DWI data files
file_path = '/content/drive/MyDrive/ISLES/dataset-ISLES22^public^unzipped^version'

for example_case in range(1,11):
    # Set images path.
    dwi_path = os.path.join(file_path, 'rawdata', 'sub-strokecase{}'.format("%04d" %example_case), 'ses-0001', 
                            'sub-strokecase{}_ses-0001_dwi.nii.gz'.format("%04d" %example_case))
    mask_path = dwi_path.replace('rawdata', 'derivatives').replace('dwi', 'msk')

# Load image data.
dwi_data = []
mask_data = []

dwi_image = nib.load(dwi_path).get_fdata()
mask_image = nib.load(mask_path).get_fdata()

# Get the shape of the DWI data from input shape construction

In [ ]:
# print the shape of the DWI data and the number of b-values/b-vectors
for i in range(1,11):
    print('DWI data shape:', dwi_image[i].shape)

# Split DWI data into train and test set

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(dwi_image, mask_image, test_size=0.2, random_state=42)


# Construction CNNs model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Reshape the data to be suitable for a CNN model
train_data = train_data.reshape((112, 72, 1))
test_data = test_data.reshape((112, 72, 1))

# Define the input shape
input_shape = train_data[0].shape

In [ ]:
# Define the input shape
input_shape = train_data[0].shape

# Define the model architecture
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))